In [ ]:
# %pip install lxml
# %pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 45.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [plotly]  WARNING: The script plotly_get_chrome is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [plotly]2m1/2 [plotly]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests as rq
import bs4
import pandas as pd
import lxml


In [6]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/145.0.0.0 Safari/537.36"}

page = rq.get(url, headers = headers)
page.status_code
len(page.text)

378272

In [3]:
page.text[0 : 200]
bs4page = bs4.BeautifulSoup(page.text, 'html.parser')
tables = bs4page.find_all('table',{'class':"wikitable"})

In [ ]:
from io import StringIO
import numpy as np
# Read the table from the StringIO object into pandas
# Note most recent version of pandas won't accept a string as input, it needs to be passed through stringio
gdp = pd.read_html(StringIO(str(tables[0])))[0]

#data cleaning - simplifying column names, getting rid of unnecessary data elements of parentheses and superscripts
gdp.rename(columns={'IMF (2026)[6]': 'IMF', 'World Bank (2024)[7]': 'WB','United Nations (2024)[8]':'UN'}, inplace=True)
gdp['Country/Territory'] = gdp['Country/Territory'].str.split('[').str[0]
gdp['IMF'] = gdp['IMF'].str.split('(').str[0]
gdp['WB'] = gdp['WB'].str.split('(').str[0]
gdp['UN'] = gdp['UN'].str.split('(').str[0]
gdp = gdp[gdp['Country/Territory']!= 'World']


#imputing IMF data first with World Bank data and then with UN data
gdp['IMF'] = gdp['IMF'].replace('—N/a', np.nan)
gdp['WB'] = gdp['WB'].replace('—N/a', np.nan)
gdp['IMF'] = gdp['IMF'].fillna(gdp['WB']).fillna(gdp['UN'])

missing_imf = gdp[gdp['IMF'].isna()]
print(len(missing_imf))
#prints 0 so the imputation worked

#gdp.head()

0


In [9]:
url_rgn = "https://en.wikipedia.org/w/index.php?title=List_of_countries_by_GDP_(nominal)&oldid=1187446467"
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/145.0.0.0 Safari/537.36"}

page_rgn = rq.get(url_rgn, headers = headers)
page_rgn.status_code
len(page_rgn.text)

407029

In [21]:
page_rgn.text[0 : 30000]
bs4page_rgn = bs4.BeautifulSoup(page_rgn.text, 'html.parser')
tables_rgn = bs4page_rgn.find_all('table')

In [34]:
gdp_rgn = pd.read_html(StringIO(str(tables_rgn[2])))[0]
gdp_rgn.head()

Country/Territory UN region IMF[1][13]            World Bank[14]             \
  Country/Territory UN region   Forecast       Year       Estimate       Year   
0             World         —  104476432       2023      100562011       2022   
1     United States  Americas   26949643       2023       25462700       2022   
2             China      Asia   17700899  [n 1]2023       17963171  [n 3]2022   
3           Germany    Europe    4429838       2023        4072192       2022   
4             Japan      Asia    4230862       2023        4231141       2022   

  United Nations[15]             
            Estimate       Year  
0           96698005       2021  
1           23315081       2021  
2           17734131  [n 1]2021  
3            4259935       2021  
4            4940878       2021

In [ ]:
gdp_rgn_subset = gdp_rgn[['Country/Territory', 'UN region']]
gdp_rgn_subset.columns = gdp_rgn_subset.columns.droplevel(0)
gdp_rgn_cmbd = gdp.merge(gdp_rgn_subset, on = 'Country/Territory', how = 'left')

#classify remaining regions which are na
print(gdp_rgn_cmbd[gdp_rgn_cmbd['UN region'].isna()])
gdp_rgn_cmbd.loc[gdp_rgn_cmbd['Country/Territory'].isin(['U.S. Virgin Islands','Saint Martin']),'UN region'] = 'Americas'
gdp_rgn_cmbd.loc[gdp_rgn_cmbd['Country/Territory'].isin(['Channel Islands','Isle of Man','Faroe Islands']),'UN region'] = 'Europe'
gdp_rgn_cmbd.loc[gdp_rgn_cmbd['Country/Territory'].isin(['Guam','Northern Mariana Islands','American Samoa']),'UN region'] = 'Oceania'

print(gdp_rgn_cmbd[gdp_rgn_cmbd['UN region'].isna()])
#now an empty data frame so all missing UN regions got assigned

            Country/Territory      IMF       WB    UN UN region
155           Channel Islands  12,508   12,508   —N/a       NaN
167               Isle of Man   7,431    7,431   —N/a       NaN
168                      Guam   6,910    6,910   —N/a       NaN
177       U.S. Virgin Islands   4,672    4,672   —N/a       NaN
179             Faroe Islands     4053     4053  —N/a       NaN
207  Northern Mariana Islands   1,096    1,096   —N/a       NaN
208            American Samoa     871      871   —N/a       NaN
210              Saint Martin     649      649   —N/a       NaN
Empty DataFrame
Columns: [Country/Territory, IMF, WB, UN, UN region]
Index: []


In [113]:
import plotly.express as px

gdp_rgn_cmbd['IMF'] = (
    gdp_rgn_cmbd['IMF']
    .astype(str)
    .str.replace(',', '', regex=False)
    .replace('nan', np.nan)
    .astype(float)
    )

fig = px.bar(gdp_rgn_cmbd, x = "UN region", y = "IMF", color = "Country/Territory")
fig.show()

In [114]:
fig.write_html("stacked_bar.html", include_plotlyjs='cdn', full_html=True)

In [94]:
## load in the hierarchy information
url = "https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/multilevel_lookup_table.txt"
multilevel_lookup = pd.read_csv(url, sep = "\t").drop(['Level5'], axis = 1)
multilevel_lookup = multilevel_lookup.rename(columns = {
    "modify"   : "roi",
    "modify.1" : "level4",
    "modify.2" : "level3",
    "modify.3" : "level2",
    "modify.4" : "level1"})
multilevel_lookup = multilevel_lookup[['roi', 'level4', 'level3', 'level2', 'level1']]
multilevel_lookup.head()

,roi,level4,level3,level2,level1
0,SFG_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L
1,SFG_R,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R
2,SFG_PFC_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L
3,SFG_PFC_R,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R
4,SFG_pole_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L


In [97]:
## Now load in the subject data
id = 127
subjectData = pd.read_csv("https://raw.githubusercontent.com/smart-stats/ds4bio_book/main/book/assetts/kirby21AllLevels.csv")
subjectData = subjectData.loc[(subjectData.type == 1) & (subjectData.level == 5) & (subjectData.id == id)]
subjectData = subjectData[['roi', 'volume']]
## Merge the subject data with the multilevel data
subjectData = pd.merge(subjectData, multilevel_lookup, on = "roi")
subjectData = subjectData.assign(icv = "ICV")
subjectData = subjectData.assign(comp = subjectData.volume / np.sum(subjectData.volume))
subjectData.head()

,roi,volume,level4,level3,level2,level1,icv,comp
0,SFG_L,12926,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.009350
1,SFG_R,10050,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R,ICV,0.007270
2,SFG_PFC_L,12783,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.009247
3,SFG_PFC_R,11507,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R,ICV,0.008324
4,SFG_pole_L,3078,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.002227


In [107]:
labels = pd.unique(
    subjectData[['level1','level2','level3']].values.ravel()
)
labels = list(labels)
labels.append('ICV')
label_indx = {name: i for i, name in enumerate(labels)}

In [108]:
sources = []
targets = []
values = []

for _, row in subjectData.iterrows():

    sources.append(label_indx['ICV'])
    targets.append(label_indx[row['level1']])
    values.append(row['volume'])
    
    sources.append(label_indx[row['level1']])
    targets.append(label_indx[row['level2']])
    values.append(row['volume'])

    sources.append(label_indx[row['level2']])
    targets.append(label_indx[row['level3']])
    values.append(row['volume'])

In [109]:
subjectData_links = pd.DataFrame({'source': sources, 'target': targets, 'value': values})
subjectData_links = subjectData_links.groupby(['source','target'], as_index=False)['value'].sum()

In [111]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = subjectData_links['source'],
      target = subjectData_links['target'],
      value = subjectData_links['value']
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [112]:
fig.write_html("sankey.html", include_plotlyjs='cdn', full_html=True)

Sankey diagram live public github webpage:
https://thomaswlewis.github.io/brain_sankey/